In [26]:
import hashlib
from sage.rings.finite_rings.integer_mod import IntegerMod_gmp
from sage.rings.polynomial.polynomial_zmod_flint import Polynomial_zmod_flint
#from math import log
# DIFINES
PRIME = 3*2**30+1
FIELD = GF(PRIME)

In [27]:
F.<x> = PolynomialRing(GF(PRIME),'x')
field_gen = FIELD.multiplicative_generator()
gen8192 = field_gen ** ((PRIME-1)/8192)
gen1024 = field_gen ** ((PRIME-1)/1024)

In [28]:
Y_INDEX = 999
a = 2 

In [29]:
def trace_calculator(a: int, trance_len: int) -> list:
    trace = [1, a]
    for i in range(2,trance_len):
        trace.append((trace[i-1] + trace[i-2])%PRIME)
    return trace

In [54]:
def polynomial_evaluation (trace: list, generator: IntegerMod_gmp) -> Polynomial_zmod_flint:
    points =[]
    for i, y in enumerate (trace): 
        points.append((generator**i, y))
    
    R = FIELD['x']
    polynomial = R.lagrange_polynomial(points)
    return polynomial

In [31]:
trance_len = 1024
trace = trace_calculator(a=a,trance_len=trance_len)
#print(trace)
poly = polynomial_evaluation(trace=trace, generator=gen1024)
# Sainaty check:
assert poly(gen1024**Y_INDEX) == trace[Y_INDEX]
Y=trace[Y_INDEX]

<class 'sage.rings.polynomial.polynomial_zmod_flint.Polynomial_zmod_flint'>


In [32]:
def compositon_polynomial(poly_list: list, random_co: dict):
    cp=0*x
    for poly in poly_list:
        cp=cp + poly * random_co[poly]
    return cp


In [33]:
def constrains_polynomials(poly: Polynomial_zmod_flint, trance_len: int, Y: IntegerMod_gmp, index_y: int , gen: IntegerMod_gmp):
    n=trance_len
    p1 = (poly-1)._divide_if_possible(x-gen**0)
    p2 = (poly - Y)._divide_if_possible(x-gen**index_y)
    
    # (x-g**0)(x-g**1)...(x-g**(n-1)) = x**n-1
    # 
    constrain_3_numer = poly(gen ** 2 * x) - poly(gen * x) - poly(x)
    # constrain_3_numer should divide by all powers of gen: x=g**0, .... x= g**(n-3)
    constrain_3_denom = (x**n-1)._divide_if_possible( (x-gen**(n-1)) * (x-gen**(n-2)) )
    # p3 = (poly(gen ** 2 * x) - poly(gen * x) - poly(x))*(x-gen**(n-1)) \
    #     * (x-gen**(n-2))*(x-gen**(n-3))/(x**n-1)
    p3 = constrain_3_numer._divide_if_possible(constrain_3_denom)
    return (p1, p2, p3)

In [34]:
def merkle(points: list):
    if len(points) <= 1:
        return points[0]
    squeezed_points = []
    for i in range(len(points)//2):
        temp_string = ''.join(str(points[i*2]))
        temp_string = temp_string.join(str(points[i*2+1]))
        squeezed_points.append(sha3(temp_string))
    if len(points)%2 == 1:
        squeezed_points.append(sha3(''.join(str(points[-1]))))
    return merkle(squeezed_points)

In [35]:
def sha3(string: str):
    byte=string.encode('ascii')
    m = hashlib.sha3_256()
    m.update(byte)
    
    return m.hexdigest()

In [36]:
def fiat_shamir_random(data: str, nonce = 0)-> int:
    if nonce:
        data = sha3(data + str(nonce))
    rand = int.from_bytes(data.encode('ascii'), "big")
    return rand%PRIME

In [37]:
points = trace_calculator(2, 15)
merkle(points)

'112bda191ff7087fc2d7fda84d713e02eb8cae32817df6a2e2a41224bf81616c'

In [38]:
def reverse_bit(n, width = 10):
    n_binary = '{:0{width}b}'.format(n, width=width)
    return int(n_binary[::-1], 2)

In [39]:
def low_degree_extension(poly: Polynomial_zmod_flint, trance_len: int, group_gen: IntegerMod_gmp, field_gen: IntegerMod_gmp):
    coset_set = [field_gen*(group_gen**i) for i in range (trance_len)]
    new_coset_set = [coset_set[reverse_bit(i, log(trance_len,2))] for i in range (trance_len)]
    return [(c, poly(c)) for c in new_coset_set]


In [40]:
def domain_extension( trace_len: int, group_gen: IntegerMod_gmp, field_gen: IntegerMod_gmp):
    coset_set = [field_gen*(group_gen**i) for i in range (trace_len)]
    new_coset_set = [coset_set[reverse_bit(i, log(trace_len,2))] for i in range (trace_len)]
    return new_coset_set

In [41]:
domain_extension(trace_len=16, group_gen=gen8192, field_gen=field_gen)

[5,
 2833855974,
 2623023147,
 2540449978,
 1056415280,
 1800233432,
 380960369,
 671626609,
 2229935889,
 2457247830,
 19433037,
 3171820980,
 2439017906,
 13901640,
 820071082,
 565703797]

In [50]:
def fri(poly: Polynomial_zmod_flint, domain: list , degree = 1024, queries = 8) -> dict:
    
    domain_length = len(domain)
    index_to_sample = [0]*queries
    proof = {} #{stage: {'root': merkle root, 'pathes':[pathes],'random':random number for naxt-stage}}
    for stage in range (log(degree, 2)):
        
        pathes = [] # [(value, path),...]
        merkel_root, merkle_layer = commit(poly, domain)
        #first stage: evaluation above the whole domain and creation of merkle tree
        if stage == 0: #qurie at queries/2 random indexes
            for i in range(0,queries,2):
                index_to_sample[i] = fiat_shamir_random(merkel_root,i)%domain_length
                index_to_sample[i+1] = index_to_sample[i]+(-1)**(index_to_sample[i]%2)
        else: #in every other stage - brings the indexes of the negative element in the domain
            new_index_to_sample = []
            for index in index_to_sample:
                index//=2
                temp_index = index + (-1)**(index%2)
                if temp_index not in new_index_to_sample:
                    new_index_to_sample.append(temp_index)
            index_to_sample = new_index_to_sample
        for index in index_to_sample:
            value_at_index, path = evaluate_points_and_path(merkle_layer, int(index))
            pathes.append((value_at_index, path))
        #second stage: takes n number randoms, and claculate n/2 time P(x_i) 1<i<n/2 + merkle path for them

        rand = fiat_shamir_random(merkel_root)
        proof[stage]= {'root':merkel_root ,'pathes':pathes,'random': rand}
        poly, domain = fri_next_layer(poly=poly, domain=domain, rand = rand)
        #third stage: calculates FRI next Layer
    
    return proof

In [43]:
def commit(poly: Polynomial_zmod_flint, domain: list):
    #first stage: evaluation above the whole domain and creation of merkle tree
    points=[(d, poly(d)) for d in domain]
    tree = MerkeTree(domain=points)
    return (tree.root, tree)

In [44]:
def fri_next_layer(poly: Polynomial_zmod_flint, domain: list , rand: int):
    #calculate the polynomial and the domain of the next stage
    even = 0*x
    odd = 0*x
    for degree,coef in poly.dict().items():
        if degree%2==0:
            even = even + coef*x**(degree//2)
        else:
            odd = odd + coef*x**(degree//2)
    next_layer = even + rand*odd
    new_domain = []
    for i in range(0,len(domain),2):
        assert domain[i]**2 == domain[i+1]**2
        new_domain.append(domain[i]**2)
    return next_layer, new_domain

In [45]:
def evaluate_points_and_path(tree: MerkeTree, index: int):
    return tree.get_value_and_path_by_index(index=index)

In [46]:
trace = []
domain = []
for i in range (1024):
    trace.append(gen1024**i)
for i in range (1024):
    domain.append(trace[reverse_bit(i, 10)])
domain[2]**2

3221225472

In [55]:
def hash_tow_elements(element1, element2):
        temp_string = ''.join(str(element1))
        temp_string = temp_string.join(str(element2))
        return(sha3(temp_string))

def hash_one_elements(element):
    return(sha3(str(element)))


class MerkeTree():
    
    tree: dict = {}
    domain_size: int
    
    def __init__(self, domain:list):
        self.tree={}
        self.domain_size = len(domain)
        # Calculate the hashes of each point in the domain.
        # Inset the leavs and their hashes to the tree. 
        domain_hashed = []
        for element in domain:
            hashed_element = hash_one_elements(element)
            self.tree[hashed_element] = element
            domain_hashed.append(hashed_element)

        # Now all the leavs are in the tree.
        # Construct the hash piramid.
        self.recursive_merkle(nodes_layer=domain_hashed)

    def recursive_merkle(self, nodes_layer: list):
        if len(nodes_layer) <= 1:
            #This is the root of the merkle tree.
            self.tree['root']=nodes_layer[0]
            return

        assert len(nodes_layer)%2 ==0
        
        # Create a new layer of nodes in the tree
        new_nodes_layre = []

        # Create a new node based on the two node beneath it.
        for i in range(len(nodes_layer)/2):
            hash_element = hash_tow_elements(nodes_layer[i*2], nodes_layer[i*2+1])
            self.tree[hash_element] = (nodes_layer[i*2], nodes_layer[i*2+1])
            new_nodes_layre.append(hash_element)
            
        return self.recursive_merkle(nodes_layer = new_nodes_layre)
    
    @property
    def root(self):
        return self.tree['root']
    
    def get_value_and_path_by_index(self, index: int):
        
        index_size = int(log(self.domain_size, 2))
        key = self.tree['root']
        
        # Shift the index from an int to a binary list.
        index_as_str = format(index, f'#0{index_size+2}b')

        # Shift from '0b1110' to '1110'
        index_as_str = index_as_str[2:] 

        path = {}
        while(index_as_str):
            value = self.tree[key]
            path[key] = value
            direction_bit = int(index_as_str[0])
            key = value[direction_bit]
            index_as_str = index_as_str[1:]
        
        #Now the key is the hash of the required index. Reauired value = tree[key] = (coset, CP[coset])
        path[key] = self.tree[key]
        return(path[key], path)


In [48]:
#domain = domain_extension(poly=CP, trance_len=16, group_gen=gen8192, field_gen=field_gen)
domain = domain_extension(trace_len=16, group_gen=gen8192, field_gen=field_gen)
root, merke_tree = commit(poly=copy, domain=domain)
#print(merke_tree)

MerkeTree is complete. self.domain_size=16


In [57]:
def prove(a: int, trace_length: int, destination, queries: int = 8):
    trace = trace_calculator(a,trace_length)
    Y = trace[destination]
    poly = polynomial_evaluation(trace, gen1024)
    p1, p2, p3 = constrains_polynomials(poly, len(trace), Y, destination, gen1024)
    domain_size = len(trace)*queries
    domain_gen = field_gen ** ((PRIME-1)/domain_size)
    domain = domain_extension(domain_size , domain_gen, field_gen)
    proof_stage_one = {} #{name:{'root' = root, 'value' = value,'path' = path}}
    random_co = {}
    merkle_p = {}
    root_p = {}
    value_at_index= {}
    path = {}
    for p in [p1, p2, p3]:
        root_p[p], merkle_p[p] = commit(p, domain)
        random_co[p] = fiat_shamir_random(root_p[p])
        
    cp = compositon_polynomial([p1, p2, p3], random_co)
    root_p[cp], merkle_p[cp] = commit(cp, domain)
    index = fiat_shamir_random(root_p[cp])%len(domain)
    polynomials = [p1, p2, p3, cp]
    names = ['p1', 'p2', 'p3', 'cp']
    for p , name in zip(polynomials, names):
        value_at_index[p], path[p] = evaluate_points_and_path(merkle_p[p], index)
        internal_proof_dict={}
        internal_proof_dict['root'] = root_p[p]
        internal_proof_dict['value'] = value_at_index[p]
        internal_proof_dict['path'] = path[p]
        proof_stage_one[name] = internal_proof_dict
    
#   return proof_stage_one
    proof_stage_two = fri(cp, domain , degree = 1024, queries = 8)
    #{stage: {'root': merkle root, 'pathes':[pathes],'random':random number for naxt-stage}}
    
    return proof_stage_one, proof_stage_two

In [58]:
proof = prove(2, 1024, 1000)
proof[1][3]['pathes']

[((951016390, 1109021098),
  {'f7d397c58baffbfe8864bd85daa1bf33d2d93e8a6472815ec58601462d54dfc4': ('da575f2e7d9e460d9b23d50bedee2530d911abcdd2ee468fb47964552940844e',
    '4d569f25fde3fb2613520a560cca832850873f0f42c4e600a1b79cf8693b13f4'),
   'da575f2e7d9e460d9b23d50bedee2530d911abcdd2ee468fb47964552940844e': ('0b21473c73b25831c854c2385a41fcde7b890df9835953518cf5879641d55c0e',
    '8d5c169abd908cf9850915ee9317ff317b712d8f219ebc0ef20e531a72370b31'),
   '8d5c169abd908cf9850915ee9317ff317b712d8f219ebc0ef20e531a72370b31': ('695aa4c4a6c632b1ee0fd7c5a2d64cf66e4589c48b9fffe4ec8a9c6a830b233d',
    '8624a9a293613dc76bdab594e374c7e30b6811695278dffe1ebf56f6cb8d86bb'),
   '695aa4c4a6c632b1ee0fd7c5a2d64cf66e4589c48b9fffe4ec8a9c6a830b233d': ('6f8663ff2f0e899888021c1ef58f32b311119e01a6eb256a44ce6221a4944e35',
    '39fd8ccb90d86756d4fb53627455dd9ef034b57660af83381ec4a80088f4c12e'),
   '39fd8ccb90d86756d4fb53627455dd9ef034b57660af83381ec4a80088f4c12e': ('1cca4bff3455e99194f6073bbd83697351b06047617bb61d

In [ ]:
class Verifier_first_stage():
    
    def __init__(self):
        """
        """
    
    def verify(self, proof_stage_one: dict, domian_size: int):
            index = self.get_index(proof_stage_one=proof_stage_one, domian_size=domian_size)
            self.verifiy_path_and_values(proof_stage_one=proof_stage_one, index=index, domian_size=domian_size)
            coeffs = self.get_coefficients(proof_stage_one=proof_stage_one)
            self.validate_cp_value_in_index(proof_stage_one=proof_stage_one, coeffs=coeffs)

    def verifiy_path_and_values(self, proof_stage_one: dict, index: int, domian_size: int):
        names = ['p1', 'p2', 'p3', 'cp']
        for name in names:
            internal_proof_dict = proof_stage_one[name]
            root = internal_proof_dict['root']
            value_at_index = internal_proof_dict['value']
            path = internal_proof_dict['path']
            self.verify_path_by_index(
                root=root, expected_value=value_at_index, index=index, domain_size=domian_size, path=path
            )
            
    def get_index(self, proof_stage_one: dict, domian_size: int):
        #print(f"{proof_stage_one['cp']=}")
        cp_tree_root = proof_stage_one['cp']['root']
        return (fiat_shamir_random(cp_tree_root)%domian_size)
    
    def get_coefficients(self, proof_stage_one:dict) -> dict:
        names = ['p1', 'p2', 'p3']
        return {name:fiat_shamir_random(proof_stage_one[name]['root']) for name in names}
    
    def validate_cp_value_in_index(self, proof_stage_one: dict, coeffs: dict):
        names = ['p1', 'p2', 'p3']
        expected_value = 0
        for name in names:
            expected_value_int = ((proof_stage_one[name]['value'])[1]*coeffs[name])
            expected_value += expected_value_int%PRIME            
        assert proof_stage_one['cp']['value'][1] == expected_value
    
    def verify_path_by_index(self, root: str, expected_value: tuple, index: int, domain_size: int, path: dict):

        index_size = int(log(domain_size, 2))
        index_as_str = format(index, f'#0{index_size+2}b')
        # shift from '0b1110' to '1110'
        index_as_str = index_as_str[2:] 
        key=root
        while(index_as_str):
            value = path[key]

            #Verify the hash:
            assert hash_tow_elements(value[0], value[1]) == key

            direction_bit = int(index_as_str[0])
            key = value[direction_bit]
            index_as_str = index_as_str[1:]

        #Now the key is the hash of the required index. Reauired value = tree[key] = (coset, CP[coset])
        value = path[key]
        assert hash_one_elements(value) == key
        assert value == expected_value
    

In [ ]:
verifier1 = Verifier_first_stage()
verifier1.verify(proof_stage_one=proof, domian_size=2048)